In [1]:
import pandas as pd
import json
import re
import pronto
import mojimoji

In [2]:
with open("../../data/ENE_compound_name.txt", 'r', encoding='utf-8') as f:
    pageid2title = {str(json.loads(line)['pageid']): json.loads(line)['title'] for line in f}
    entry_title = list(pageid2title.values())

In [4]:
train_sentence_df = pd.read_csv("../../data/Production_train_sentence_with_repl_compound.csv")
train_sentence_df.head()

,_id,label,repl_sentence,sentence,title
0,10166,False,[title-compound] (英: [title-compound]) は分子式が N...,アンモニア (英: ammonia) は分子式が NH 3 で表される無機化合物。,アンモニア
1,10166,False,常温常圧では無色の気体で、特有の強い刺激臭を持つ。,常温常圧では無色の気体で、特有の強い刺激臭を持つ。,アンモニア
2,10166,False,水に良く溶けるため、水溶液（[compound]）として使用されることも多く、化学工業では基...,水に良く溶けるため、水溶液（アンモニア水）として使用されることも多く、化学工業では基礎的な窒...,アンモニア
3,10166,False,塩基の程度は[compound]より弱い。,塩基の程度は水酸化ナトリウムより弱い。,アンモニア
4,10166,False,窒素原子上の孤立電子対のはたらきにより、金属錯体の配位子となり、その場合はアンミンと呼ばれる。,窒素原子上の孤立電子対のはたらきにより、金属錯体の配位子となり、その場合はアンミンと呼ばれる。,アンモニア


In [5]:
flatten = lambda items: [i for sublist in items if sublist for i in sublist]

def get_asynonyms_from_wikidata(entry):
    '''
    arg: json of WikiData entry
    return: English and Japanese synonyms list
    '''
    synonyms = []
    
    aliases_dict = list(entry['entities'].values())[0]['aliases']
    if aliases_dict and aliases_dict.get('en'):
        synonyms += [alias['value'] for alias in aliases_dict.get('en')]
    if aliases_dict and aliases_dict.get('ja'):
        synonyms += [alias['value'] for alias in aliases_dict.get('ja')]
    if get_title_from_wikidata(entry, lang='en'):
        synonyms += [get_title_from_wikidata(entry, lang='en')]
    
    return synonyms

def get_title_from_wikidata(entry, lang='ja'):
    label_dict = list(entry['entities'].values())[0]['labels']
    if label_dict.get(lang):
        return label_dict.get(lang)['value']
    return None

def get_CID(article: dict) -> str:
    if article.get('Record') and article.get('Record').get('RecordNumber'):
        return str(article.get('Record').get('RecordNumber'))
    else:
        return None
    
def get_synonyms_from_PubChem(article):
    synonym_list = \
    [synonyms['StringValueList']
     for section in article['Record']['Section'] if section['TOCHeading'] == 'Names and Identifiers'
     for sub_section in section['Section'] if sub_section['TOCHeading'] == 'Synonyms'
     for synonym_info in sub_section['Section'] if synonym_info['TOCHeading'] == 'MeSH Entry Terms'
     for synonyms in synonym_info['Information']
    ]
    
    return flatten(synonym_list)

def append_dict(dic, key, data):
    if not isinstance(data, list):
        data = [data]
    
    _dic = dic.copy()
    if _dic.get(key) is None:
        _dic[key] = data
    elif isinstance(_dic.get(key), list):
        _dic[key] += data
    else:
        _dic[key] = [_dic[key]] + data
        
    return _dic

def remove_disambiguation(word):
    return re.sub(r'\s\(.+\)$|\s（.+）$', '', word)

def reverse_table(table: dict):
    if not isinstance(list(table.values())[0], list):
        return {v: k for k, v in table.items()}
    
    new_table = {}
    for k, values in table.items():
        for v in values:
            new_table = append_dict(new_table, v, k)
                
    return new_table

## 同義語辞書作成

In [6]:
# 同義語辞書 {title: synonyms}
synonyms_dict = {}

In [7]:
# WikiDataから同義語データを取得
with open("../../data/compound_wikidata.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
        entry = json.loads(line)
        title = get_title_from_wikidata(entry)
        if title is None:
            continue
        synonyms = get_asynonyms_from_wikidata(entry)
        synonyms_dict = append_dict(synonyms_dict, title, synonyms)

In [ ]:
# ChEBI Ontology から同義語データを取得
ont = pronto.Ontology("../../data/chebi.owl")

with open("../../data/pageid2ChEBI.json", 'r') as f:
    pageid2ChEBI_table = json.load(f)

ChEBI2pageid_table = reverse_table(pageid2ChEBI_table)

for entry in ont:    
    pageid_list = ChEBI2pageid_table.get(entry.id)
    if pageid_list is None:
        continue
    
    synonyms = list(entry.synonyms)
    if len(synonyms) is 0:
        continue
    synonyms = [synonym.desc for synonym in synonyms]
    
    for pageid in pageid_list:
        title = remove_disambiguation(pageid2title.get(pageid))
        synonyms_dict = append_dict(synonyms_dict, title, synonyms)

In [107]:
# PubChem から同義語データを取得
with open("../../data/pageID2CID_using_wikidata.json", 'r') as f:
    cid2pageid_table = reverse_table(json.load(f))

with open("../../data/pubchem_articles.jsonl", 'r') as f:
    for line in f:
        entry = json.loads(line)
        
        cid = get_CID(entry)
        if cid is None: continue
        
        pageid_list = cid2pageid_table.get(cid)
        if pageid_list is None: continue
            
        synonyms = get_synonyms_from_PubChem(entry)
        for pageid in pageid_list:
            title = remove_disambiguation(pageid2title.get(pageid))
            synonyms_dict = append_dict(synonyms_dict, title, synonyms)

## 置換

In [186]:
def escape(s):
    _s = s.replace(r'(', '\(').replace(r')', '\)')
    _s = _s.replace(r'[', '\[').replace(r']', '\]')
    _s = _s.replace(r'{', '\{').replace(r'}', '\}')
    _s = _s.replace(r'-', '\-')
    _s = _s.replace(r'+', '\+')
    _s = _s.replace(r'^', '\^')
    _s = _s.replace(r'.', '\.')
    _s = _s.replace(r'|', '\|')
    _s = _s.replace(r'?', '\?')
    _s = _s.replace(r'*', '\*')
    
    return _s

In [4]:
def replace_compound(matchobj, title=None):
    synonyms = synonyms_dict.get(title)
    if synonyms is None:
        synonyms = [title]
    else:
        synonyms += [title]
    
    if matchobj.group(0) in synonyms:
        return "[title-compound]"
    return "[compound]"

In [8]:
# 日化辞辞書読み込み
nikkaji_compounds = pd.read_csv("../../data/mecab_nikkaji.csv", encoding='cp932')[['Surface form']].rename(columns={'Surface form': 'name'})
## 全角英数字を半角に
nikkaji_compounds.name = nikkaji_compounds.name.apply(lambda x: mojimoji.zen_to_han(x, kana=False))

In [166]:
compound_name_list = [remove_disambiguation(compound) for compound in entry_title]
compound_name_list += nikkaji_compounds.name.tolist()
compound_name_list += flatten(list(synonyms_dict.values()))
compound_name_list = list(set(compound_name_list))

compound_name_list.sort(key=len)
for i, name in enumerate(compound_name_list):
    if len(name) > 1:
        compound_name_list = compound_name_list[i:]
        break
compound_name_list.remove("生産")

compound_name_list = [escape(compound) for compound in compound_name_list]
compound_name_list.sort(key=len, reverse=True)

r = re.compile('|'.join(compound_name_list))

In [192]:
%%time
replace_compound_train_df = train_sentence_df.copy()
replace_compound_train_df['repl_sentence'] = replace_compound_train_df.apply(
    lambda x: r.sub(lambda matchobj: replace_compound(matchobj, title=x.title[0]), x.sentence)
    , axis=1
)

CPU times: user 5min 45s, sys: 56 ms, total: 5min 45s
Wall time: 5min 45s


In [198]:
replace_compound_train_df.to_csv("../../data/Production_train_sentence_with_repl_compound.csv", index=False)

In [196]:
for label, g in replace_compound_train_df.groupby('label'):
    count_sentence = len(g)
    isin_compound = g.repl_sentence.str.contains(r'\[compound\]').sum()
    print(label, "\t in compound rate:", isin_compound / count_sentence)

False 	 in compound rate: 0.45214599928664845
True 	 in compound rate: 0.8248299319727891


In [197]:
for label, g in replace_compound_train_df.groupby('label'):
    count_sentence = len(g)
    isin_compound = g.repl_sentence.str.contains(r'\[title\-compound\]').sum()
    print(label, "\t in title-compound rate:", isin_compound / count_sentence)

False 	 in title-compound rate: 0.3746284627273808
True 	 in title-compound rate: 0.39285714285714285


In [190]:
true_df = replace_compound_train_df[replace_compound_train_df.label == True]
true_df[~true_df.sentence.str.contains(r'\[compound\]')].values

array([[10166, True, '現在では[title-compound]の工業生産はハーバー・ボッシュ法によるものが一般的である。',
        'アンモニア'],
       [10166, True,
        '実際のプラントでは水素と窒素を鉄触媒存在下 25〜35 MPa、約500℃ で反応させると、 N 2 + 3 H 2 ⟶ 2 NH 3 の反応によって[title-compound]が生成する。',
        'アンモニア'],
       [10166, True,
        'C a O → 2000 ∘ C 3 C C a C 2 → 1000 ∘ C N 2 C a C N 2 → 3 H 2 O 2 N H 3 ルテニウム触媒（Ru-活性炭-K） 尾崎、秋鹿らによるハーバー法よりも温和な条件で[title-compound]を合成できるルテニウム触媒を用いた合成法。',
        'アンモニア'],
       [10166, True,
        'C12A7 Electride アルミナセメントの構成成分を用いる方法で、常圧 320〜400℃で合成可能。',
        'アンモニア'],
       [11006, True, '炭化アルミニウムに室温で水を反応させて加水分解する。', 'メタン'],
       [12437, True,
        '現在市場に出回っている[title-compound]は、大部分がアルコール発酵によって製造されている。', 'エタノール'],
       [27129, True, '例としてBASF社の、有機塩基を用いて抽出するという手法が挙げられる。', 'ギ酸'],
       [27129, True,
        '1671年、イギリスの博物学者であるジョン・レイ (John Ray) が、大量の死んだアリの蒸留により[title-compound]を初めて単離し、「アリの酸 ([title-compound])」と命名した。',
        'ギ酸'],
       [27347, True,
        '工業的な酸化方法では、銅などの触媒を用いてアルコールを空気または酸素で酸化する方法がよく用いられ

In [191]:
false_df = replace_compound_train_df[replace_compound_train_df.label == False]
false_df[false_df.sentence.str.contains(r'\[title\-compound\]')]

,_id,label,sentence,title
0,10166,False,[title-compound] (英: [title-compound]) は分子式が N...,アンモニア
6,10166,False,ラテン語の sol [title-compound]um（アモンの塩）を語源とする。,アンモニア
8,10166,False,[title-compound]を初めて合成したのはジョゼフ・プリーストリー（1774年）である。,アンモニア
10,10166,False,[title-compound]分子は窒素を中心とする四面体構造を取っており、各頂点には3つ...,アンモニア
18,10166,False,[title-compound]は液化しやすく、20℃では、0.857 MPa （8.46気...,アンモニア
19,10166,False,液体[title-compound]の性質は水と似ている。,アンモニア
20,10166,False,例えば、さまざまな物質を溶解し、液体[title-compound]自体も水溶液と似た性質を...,アンモニア
21,10166,False,液体[title-compound]の蒸発潜熱は1268 J g-1（0℃）であり、この値は...,アンモニア
22,10166,False,液体[title-compound]中では弱い自己解離が存在し、-33℃（沸点）におけるイオ...,アンモニア
23,10166,False,"2 NH 3 ↽ − − ⇀ ( NH 4 + ) + NH 2 − , K s = 10 ...",アンモニア
